In [18]:
from keras.applications import ResNet50, VGG16, InceptionV3, EfficientNetB0
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

**Loading ResNet50 Model**

In [ ]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(75, 75, 3))

In [19]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 75, 75, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 81, 81, 3)            0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 38, 38, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 38, 38, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                       

**Loading VGG16 Model**

In [3]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(75, 75, 3))

In [20]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 75, 75, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 75, 75, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 75, 75, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 37, 37, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 37, 37, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 37, 37, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 18, 18, 128)       0     

**Loading InceptionV3 Model**

In [4]:
inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(75, 75, 3))

In [21]:
inception_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 75, 75, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 37, 37, 32)           864       ['input_3[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 37, 37, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 37, 37, 32)           0         ['batch_normalizati

**Loading EfficientNetB0 Model**

In [5]:
EfficientNetB0 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(75, 75, 3))

**Freezing Cov-base of every model**

In [6]:
# Freeze the layers of pre-trained models
for model in [resnet_model, vgg_model, inception_model, EfficientNetB0]:
    model.trainable = False

**Architecture using ResNet50 Model**

In [7]:
model_resnet = Sequential(
    [resnet_model, 
     Flatten(), 
     Dense(units=256, activation='relu'), 
     Dense(units=3, activation='softmax')]
)

**Architecture using VGG16 Model**

In [8]:
model_vgg = Sequential(
    [vgg_model, 
     Flatten(), 
     Dense(units=256, activation='relu'), 
     Dense(units=3, activation='softmax')]
)

**Architecture using InceptionV3 Model**

In [9]:
model_inception = Sequential(
    [inception_model, 
     Flatten(), 
     Dense(units=256, activation='relu'), 
     Dense(units=3, activation='softmax')]
)

**Architecture using EfficientNetB0 Model**

In [10]:
EfficientNetB0 = Sequential(
    [EfficientNetB0, 
     Flatten(), 
     Dense(units=256, activation='relu'), 
     Dense(units=3, activation='softmax')]
)

**Image Pre-Processing**

In [11]:
# Train individual models with ImageDataGenerator
training_data_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

testing_data_generator = ImageDataGenerator(
    rescale=1.0/255
)

training_dataset = training_data_generator.flow_from_directory(
    'Covid-19/train',
    target_size=(75, 75),
    batch_size=32,
    class_mode='categorical'
)

testing_dataset = testing_data_generator.flow_from_directory(
    'Covid-19/test',
    target_size=(75, 75),
    batch_size=32,
    class_mode='categorical'
)

Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


**Training of All models**

In [12]:
# Train individual models
models = [model_resnet, model_vgg, model_inception, EfficientNetB0]
model_accuracies = {}

for model in models:
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(training_dataset, epochs=15, verbose=1)

    # Evaluate on testing set
    loss, accuracy = model.evaluate(testing_dataset, verbose=0)
    model_accuracies[model.name] = accuracy
    print(f"{model.name} - Test Accuracy: {accuracy}")

best_model_name = max(model_accuracies, key=model_accuracies.get)
best_model_accuracy = model_accuracies[best_model_name]
print(f"\nBest Model: {best_model_name}, Accuracy: {best_model_accuracy}")


Epoch 1/15
8/8 [==============================] - 8s 675ms/step - loss: 1.6752 - accuracy: 0.4263
Epoch 2/15
8/8 [==============================] - 6s 686ms/step - loss: 1.1005 - accuracy: 0.5020
Epoch 3/15
8/8 [==============================] - 5s 648ms/step - loss: 0.7464 - accuracy: 0.6494
Epoch 4/15
8/8 [==============================] - 6s 717ms/step - loss: 0.6416 - accuracy: 0.6972
Epoch 5/15
8/8 [==============================] - 5s 667ms/step - loss: 0.5989 - accuracy: 0.7570
Epoch 6/15
8/8 [==============================] - 6s 682ms/step - loss: 0.5791 - accuracy: 0.7769
Epoch 7/15
8/8 [==============================] - 6s 672ms/step - loss: 0.5342 - accuracy: 0.7649
Epoch 8/15
8/8 [==============================] - 6s 727ms/step - loss: 0.5853 - accuracy: 0.7530
Epoch 9/15
8/8 [==============================] - 5s 679ms/step - loss: 0.5509 - accuracy: 0.7570
Epoch 10/15
8/8 [==============================] - 6s 701ms/step - loss: 0.5718 - accuracy: 0.7729
Epoch 11/15
8/8 [==

In [13]:
# Make predictions on testing set using the best model
best_model = [model for model in models if model.name == best_model_name][0]

In [26]:
print(f'Best Pre-Trained Model is:- {best_model} , {best_model.name}')

Best Pre-Trained Model is:- <keras.src.engine.sequential.Sequential object at 0x0000026599177550> , sequential_1


In [17]:
best_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 256)               524544    
                                                                 
 dense_3 (Dense)             (None, 3)                 771       
                                                                 
Total params: 15240003 (58.14 MB)
Trainable params: 525315 (2.00 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [14]:
ensemble_pred = best_model.predict(testing_dataset)
ensemble_classes = ensemble_pred.argmax(axis=-1)

3/3 [==============================] - 1s 256ms/step


In [15]:
# Display classification report and confusion matrix for the best model
print("Classification Report:\n", classification_report(testing_dataset.classes, ensemble_classes))
print("Confusion Matrix:\n", confusion_matrix(testing_dataset.classes, ensemble_classes))

Classification Report:
               precision    recall  f1-score   support

           0       0.35      0.35      0.35        26
           1       0.20      0.20      0.20        20
           2       0.35      0.35      0.35        20

    accuracy                           0.30        66
   macro avg       0.30      0.30      0.30        66
weighted avg       0.30      0.30      0.30        66

Confusion Matrix:
 [[ 9 10  7]
 [10  4  6]
 [ 7  6  7]]
